In [ ]:
# Check geemap installation
import subprocess

try:
    import geemap
except ImportError:
    print('geemap package is not installed. Installing ...')
    subprocess.check_call(["python", '-m', 'pip', 'install', 'geemap'])

geemap package is not installed. Installing ...


In [ ]:
# Import libraries
import os
import ee
import ee.mapclient
import geemap
import ipywidgets as widgets
from bqplot import pyplot as plt
from ipyleaflet import WidgetControl

In [1]:
#ee.Authenticate()

In [ ]:
ee.Initialize()

### Getting the region of interest 
1. boundary of the Lake using the line tool 
2. use draw_last_feature.geometry() to get the region from the map and get it's info for coordinates 

In [ ]:
Map = geemap.Map()
Map

Map(center=[20, 0], controls=(WidgetControl(options=['position', 'transparent_bg'], widget=HBox(children=(Togg…

In [ ]:
# Draw any shapes on the map using the Drawing tools before executing this code block -> Border 
feature = Map.draw_last_feature
roi = feature.geometry()
roi.getInfo()

{'coordinates': [[[69.656067, 22.375476],
   [69.656067, 23.104997],
   [70.515747, 23.104997],
   [70.515747, 22.375476],
   [69.656067, 22.375476]]],
 'geodesic': False,
 'type': 'Polygon'}

Just to store the initial lake co-ordinates output : 
{'coordinates': [[69.662268, 23.254555],
  [69.663255, 23.254398],
  [69.66495, 23.254969],
  [69.666152, 23.25493],
  [69.668019, 23.253511],
  [69.667525, 23.252249],
  [69.666753, 23.251086],
  [69.666646, 23.25008],
  [69.665422, 23.248247],
  [69.663813, 23.249035],
  [69.663427, 23.248444],
  [69.663835, 23.248089],
  [69.660509, 23.245526],
  [69.65847, 23.247616],
  [69.658535, 23.248503],
  [69.660358, 23.250297],
  [69.660981, 23.250593],
  [69.660959, 23.251046],
  [69.661582, 23.251815],
  [69.662054, 23.252269],
  [69.662569, 23.253471],
  [69.662161, 23.254615]],
 'geodesic': False,
 'type': 'LineString'}

In [ ]:
IMG = ee.ImageCollection("COPERNICUS/S2").filterBounds(roi).sort('CLOUDY_PIXEL_PERCENTAGE',True).first().clip(roi)

Adding the border as a layer on the map so that it is always visualised whenever we print the map

In [ ]:
Map.addLayer(roi, {}, "Selected region")
Map

Map(bottom=57282.0, center=[22.413567638369805, 70.64208984375001], controls=(WidgetControl(options=['position…

In [ ]:
#IMG = ee.ImageCollection("COPERNICUS/S2_SR").filterBounds(roi).sort('CLOUDY_PIXEL_PERCENTAGE',True).first().clip(roi)

def ndwi_f(img):
  #water mask
  ndwi = img.normalizedDifference(['B3', 'B8']).rename('NDWI');
  return img.addBands(ndwi).updateMask(ndwi.gt(0))

def mndwi_f(img):
  #water mask
  mndwi1 = img.normalizedDifference(['B3', 'B11']).rename('MNDWI1');
  return img.addBands(mndwi1).updateMask(mndwi1.gt(0))

def ndci_f(img):
  #water mask
  ndci = img.normalizedDifference(['B5', 'B4']).rename('NDCI');
  return img.addBands(ndci)

IMG_water = ndwi_f(IMG)
IMG_water = mndwi_f(IMG_water)
IMG_NDCI = ndci_f(IMG_water)

viz = {'min':0.1,'max':0.4,'palette':['cyan','orange','red']}
Map.addLayer(IMG,{'bands':['B4','B3','B2'],min:0,max:3500},'IMG')
Map.addLayer(IMG_water.select('NDWI'),{'palette':['cyan']},"IMG_water_NDWI")
Map.addLayer(IMG_water.select('MNDWI1'),{'palette':['cyan']},"IMG_water_MNDWI1")
Map.addLayer(IMG_NDCI.select('NDCI'),viz,"IMG_NDCI")



In [ ]:
Map

Map(bottom=57282.0, center=[22.862256424791948, 430.50750732421875], controls=(WidgetControl(options=['positio…

In [ ]:
ee.Image(IMG.get(0))